In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cd /content/drive/MyDrive/6240_project/project_data/

In [1]:
import gzip
import json
import re
import os
import sys
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import time
from collections import defaultdict
from tqdm import tqdm
import pickle


In [2]:
def load_data(file_name):
    count = 0
    data = []
    with gzip.open(file_name) as fin:
        for l in fin:
            d = json.loads(l)
            count += 1
            data.append(d)
            
          
    return data

In [3]:
interactions = load_data('datasets/goodreads_interactions_poetry.json.gz')

In [4]:
interactions_df = pd.json_normalize(interactions)

In [5]:
len(interactions_df)

2734350

In [6]:
user_map = pd.read_csv('datasets/user_id_map.csv')
book_map = pd.read_csv('datasets/book_id_map.csv')

In [7]:
rating_df = interactions_df.merge(user_map,on='user_id')[['user_id_csv','book_id','rating']]

In [8]:
non_zero = rating_df[rating_df['rating']!=0].reset_index(drop=True)
non_zero

,user_id_csv,book_id,rating
0,0,1384,4
1,0,1376,4
2,0,30119,5
3,1,30119,3
4,2,240007,4
...,...,...,...
1229054,876132,2547,5
1229055,876133,11047097,4
1229056,876144,7433930,5
1229057,876144,16170625,5


In [13]:
non_zero.to_csv("user_item_ratings.csv",index = False)

In [9]:
def preprocessing(training_data):
  """
  Arguments:
  training data (str-type numpy.array): the training data containing user_id, movie_id, and normalized rating (0-1) information.

  Returns:
  R_ui (dictionary of dictionaries): this dictionary contains rating information of each user. 
   - The key is user_id (string), the value is a dictionary whose key is item_id (string) and value is rating (float).
   - Thus, R_ui['1']['260'] = 1.0. R_ui should be computed with training data.
  R_iu (dictionary of dictionaries): it is similar to R_ui, but the key of a dictionary is item_id (string). 
   - Thus, R_ui['260']['1'] = 1.0. R_iu should be computed with training data.

  Steps:
  1. for each training example with (user u, item i, and rating r), R_ui[u][i] should be r (float). R_iu can be computed similarly.
  """
  R_ui,R_iu = defaultdict(dict),defaultdict(dict)

  ## Add code below [0.5 points] ##

  for index in tqdm(range(len(training_data)),desc="Complete..."):
    
    u_id = str(training_data.loc[index]['user_id_csv'])
    it_id = str(training_data.loc[index]['book_id'])
    r = float(training_data.loc[index]['rating'])

    R_ui[u_id][it_id] = r
    R_iu[it_id][u_id] = r

  
  #################################
  return R_ui,R_iu

In [11]:
# R_ui,R_iu = preprocessing(non_zero)

In [12]:
# print('R_ui of user 0 and item 1384 = {}'.format(R_ui["0"]['1384']))
# print('R_iu of user 30119 and item 1 = {}'.format(R_iu['30119']['1']))

In [ ]:
def gradient_descent_update(U,V,K):
  """
  Do not modify this function. There is a -2.0 point penalty if you modify this function.

  Arguments: 
  U,V (dictionary of numpy.array): current user and item profile dictionaries. 
   - The key is either user_id or item_id, and the value is the corresponding user or item profile (numpy.array; dim:K).
  K (int): the number of latent factors.

  Returns:
  Updated U,V (dictionary of numpy.array): updated user and item profile dictionaries. 
   - The key is either user_id or item_id, and the value is the corresponding user or item profile (numpy.array; dim:K).
  """
  mu = 0.00001
  lambda_value = 0.00001
  for user in U.keys():
    updates = np.zeros(K)
    for item in R_ui[user].keys():
      pred = np.inner(U[user],V[item])
      error = R_ui[user][item] - pred
      updates += error*V[item]
    final_updates = 2*mu*updates - 2*lambda_value*U[user]
    U[user] += final_updates

  for item in V.keys():
    updates = np.zeros(K)
    for user in R_iu[item].keys():
      pred = np.inner(U[user],V[item])
      error = R_iu[item][user] - pred
      updates += error*U[user]
    final_updates = 2*mu*updates - 2*lambda_value*V[item]
    V[item] += final_updates
  return U,V

In [ ]:
def matrix_factorization (training_data, K=15, epochs = 100):
  """
  Arguments:
  training data (str-type numpy.array): the training data containing user_id, movie_id, and normalized rating (0-1) information.
  K (int): number of latent factors used for matrix factorization.
  epochs (int): number of repetitions of the updates of U and V.

  Returns:
  U,V (dictionary of float-type numpy.array): learned user and item profile dictionaries. 
   - The key is either user_id or item_id, and the value is the corresponding user or item profile (float-type numpy.array; dim:K).

  Steps for the first code block:
  1. compute the maximum value using 'sqrt(avg(ratings of all training examples)/K)' for the initialization (ratings of all training examples should be float-type, not str-type here).
  2. for each user u in training_data, initialize the value of U[u] with a size-K numpy.array (float) filled with random values between 0 the maximum value.
  3. initialize V[v] for each item v in training_data like step 2.
      - when you assign the initial value, please use R_ui.keys(), R_iu.keys() to keep the order and to avoid multiple initialization.
  """

  np.random.seed(0)
  U,V = defaultdict(np.array),defaultdict(np.array)

  ## Add code below [1.0 points] ##

  avg_rating = np.mean(training_data[:,2].astype(float))

  max_value = (avg_rating/K)**0.5

  users = list(R_ui.keys())
  items = list(R_iu.keys())

  for user in users:
    U[user] = np.random.uniform(low=0,high=max_value,size=K).astype(float)

  for item in items:
    V[item] = np.random.uniform(low=0,high=max_value,size=K).astype(float)

  #################################

  """
  Steps for the second code block:
  1. for each iteration, call the gradient_descent_update with current U and V.
  2. update the user and item profile matrices with the returned U and V.
  """
  ## Add code below [1.0 points] ##

  for j in tqdm(range(epochs),desc='Completed......'):
    U,V = gradient_descent_update(U,V,K)

  #################################
  return U,V

In [ ]:
# with open('user_dict.pickle', 'wb') as handle:
#     pickle.dump(R_ui, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('item_dict.pickle', 'wb') as handle:
#     pickle.dump(R_iu, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
training_data = np.array(non_zero,dtype=np.float64)

In [ ]:
(W,H) = matrix_factorization(training_data)

Completed......: 100%|██████████| 100/100 [35:30<00:00, 21.30s/it]


In [ ]:
with open('user1.pickle', 'wb') as handle:
    pickle.dump(W, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('item1.pickle', 'wb') as handle:
    pickle.dump(H, handle, protocol=pickle.HIGHEST_PROTOCOL)